<a href="https://colab.research.google.com/github/JTEC-Iritani/ICT-Training-2021/blob/main/Practice_17_Scraping_iHydro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Data from iHydro

In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import pandas_profiling
import numpy as np

In [ ]:
# BeautifulSoupで日本語使用のため
!pip install chardet
font = {"family": "TakaoGothic"}

In [ ]:
# Pandas Profilingのインストール（Google Colabでは毎回実行）
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git

In [ ]:
!pip show pandas_profiling

In [ ]:
import warnings
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

warnings.filterwarnings('ignore')

In [ ]:
# 雨量データ蓄積用DataFrameの作成・初期化（1回目のみ）
cols_f = ["STATION", "DIVISION", "BASIN", "DATE", "RAINFALL"]
df = pd.DataFrame(index=[], columns=cols_f) # データ蓄積用

In [ ]:
response = requests.get('https://ihydro.sarawak.gov.my/iHydro/en/datatable/rainfall/latest-rainfall.jsp?division_id=1&page=1') # ウェブページデータの取得 Division = Kuching, Page = 1
# response = requests.get('https://ihydro.sarawak.gov.my/iHydro/en/datatable/rainfall/latest-rainfall.jsp?division_id=1&page=2') # ウェブページデータの取得 Division = Kuching, Page = 2
# response = requests.get('https://ihydro.sarawak.gov.my/iHydro/en/datatable/rainfall/latest-rainfall.jsp?division_id=1&page=3') # ウェブページデータの取得 Division = Kuching, Page = 3

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser') # データの整形

In [ ]:
soup.find(class_='header-text').text # ヘッダの内容確認

In [ ]:
# 雨量データ用DataFrameの作成・初期化
cols_f = ["STATION", "DIVISION", "BASIN", "DATE", "RAINFALL"]
df_rain_data_f = pd.DataFrame(index=[], columns=cols_f) # データ蓄積用
df_rain_data_row1  = pd.DataFrame(index=[], columns=cols_f) # 作業用
cols_w = ["NUM", "STATION","DIVISION", "BASIN", "LAST", "D1", "D2", "D3", "D4", "D5", "D6", "D7"]
df_rain_table_w = pd.DataFrame(index=[], columns=cols_w) # 作業用

In [ ]:
rain_table = soup.find_all(class_='table-bordered table-hover') # 表データの取得
# rain_data = soup.find_all(class_='table-content')

In [ ]:
rain_data_loc = soup.find_all(class_= "table-content-hyperlink") # Station Nameの取得

In [ ]:
num_rows = len(rain_data_loc) #データ行数の取得

In [ ]:
num_rows #データ行数の表示（確認用）

In [ ]:
rain_data_loc[0].text

In [ ]:
# Station Nameを作業用DataFrameにセット
for r_idx in range(num_rows):
    df_rain_table_w.at[r_idx, "STATION"]  = rain_data_loc[r_idx].text

In [ ]:
df_rain_table_w = df_rain_table_w.apply(lambda d: d.str.strip()) # 制御コード削除

In [ ]:
df_rain_table_w # 作業用DataFrame表示（確認用）

In [ ]:
rain_data = soup.find_all(class_= "table-content") # 雨量データの取得

In [ ]:
rain_data[5].text

In [ ]:
# 場所名の情報、雨量データ等を作業用DataFrameにセット
for r_idx in range(num_rows):
    df_rain_table_w.at[r_idx, "NUM"]  = rain_data[r_idx * 11].text
    df_rain_table_w.at[r_idx, "DIVISION"]  = rain_data[r_idx * 11 + 1].text
    df_rain_table_w.at[r_idx, "BASIN"]  = rain_data[r_idx * 11 + 2].text
    df_rain_table_w.at[r_idx, "LAST"]  = rain_data[r_idx * 11 + 3].text
    df_rain_table_w.at[r_idx, "D1"]  = rain_data[r_idx * 11 + 4].text
    df_rain_table_w.at[r_idx, "D2"]  = rain_data[r_idx * 11 + 5].text
    df_rain_table_w.at[r_idx, "D3"]  = rain_data[r_idx * 11 + 6].text
    df_rain_table_w.at[r_idx, "D4"]  = rain_data[r_idx * 11 + 7].text
    df_rain_table_w.at[r_idx, "D5"]  = rain_data[r_idx * 11 + 8].text
    df_rain_table_w.at[r_idx, "D6"]  = rain_data[r_idx * 11 + 9].text
    df_rain_table_w.at[r_idx, "D7"]  = rain_data[r_idx * 11 + 10].text

In [ ]:
date_table_w = soup.find_all(class_= "caption-description-center") # カラム名から日付データの取得

In [ ]:
date_table_w

In [ ]:
date_table_w[7].text

In [ ]:
# リストの初期化
date_w = [0] * 6

In [ ]:
# 日付のセット
for idx in range(6):
    date_w[idx] = date_table_w[idx + 7].text

In [ ]:
date_w

In [ ]:
# 出力用フォーマットに変換
for idx1 in range(num_rows):
  for idx2 in range(6):
    df_rain_data_row1.at[0, "STATION"] = df_rain_table_w.at[idx1, "STATION"]
    df_rain_data_row1.at[0, "DIVISION"] = df_rain_table_w.at[idx1, "DIVISION"]
    df_rain_data_row1.at[0, "BASIN"] = df_rain_table_w.at[idx1, "BASIN"]
    df_rain_data_row1.at[0, "DATE"] = date_w[idx2]
    df_rain_data_row1.at[0, "RAINFALL"] = df_rain_table_w.iat[idx1, idx2 + 5]
    df_rain_data_f = df_rain_data_f.append(df_rain_data_row1)


In [ ]:
df_rain_data_f['DATE'] = pd.to_datetime(df_rain_data_f['DATE'], format = '%d-%m-%Y') # 文字列から日付形式に変換

In [ ]:
df_rain_data_f['RAINFALL'] = df_rain_data_f['RAINFALL'].astype(float) # 文字列から数値に変換

In [ ]:
df_rain_data_f.tail(12)

In [ ]:
# 属性の確認
df_rain_data_f.info()

In [ ]:
# 2回目以降実行
# ローカルファイル（既存のクチン雨量データを蓄積したもの）をアップロードする（Google Colaboratoryの場合）
# The Littlest JupyterHubの場合：下記の2行は実行不要。必要に応じ、別途マニュアルでCSVファイルをアップロードしておく
from google.colab import files
uploaded = files.upload()

In [ ]:
# 2回目以降実行
# pickleデータをDataFrameに変換（フォルダの場所を確認し、パスを必要に応じ修正すること）
# df = pd.read_pickle('../data/Kuching_Rain_Data_202109_1.pickle') # JupyterLab等用
df = pd.read_pickle('Kuching_Rain_Data_202109_1.pickle') # Google Colaboratory用

In [ ]:
# 既存の蓄積データへの追加
df = df.append(df_rain_data_f)

In [ ]:
df.drop_duplicates(inplace=True) # 重複データの削除

In [ ]:
df.info()

In [ ]:
df.reset_index(inplace=True, drop=True) # インデックスのリセット

In [ ]:
df.to_pickle("Kuching_Rain_Data_202109_1.pickle") # DataFrameをpickle形式に変換

In [ ]:
# ファイルをダウンロードする（Google Colaboratoryの場合）
from google.colab import files
files.download("Kuching_Rain_Data_202109_1.pickle")